In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMZN', 'ANET',
    'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD', 'DDOG', 'DOCS',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED', 'GOOG',
    'IDXX', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU', 'MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'OKTA', 'OPEN', 'PAC', 'PANW',
    'PAYC', 'PINS', 'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN',
    'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC', 'TEAM',
    'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD',
    'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AMEH', 'BYRN', 'FCPT', 'FCUV', 'INTZ', 'MNMD', 'RVP', 'ZYXI']
BEST_WEIGHTED = [
    'AMRK', 'APT', 'CELH', 'EP', 'GBOX', 'GCBC', 
    'GME', 'IEI', 'MCVT', 'MICS', 'OMQS', 'ORLA', 'POWW', 'PRPH',
    'SBOW', 'VIRT', 'VIVO', 'WAVD', 'WKHS']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [10]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [11]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,629.48
ABMD,564.56
ABNB,370.44
ADYEY,4646.9
AMD,1879.9


In [12]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,629
ABMD,565
ABNB,370
ADYEY,4647
AMD,1880


In [13]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [14]:
et_df.loc[et_df.ET == '--']
#et_df.loc['NVCR', 'ET'] = 79.24

,ET


In [15]:
check_symbols(et_df)

All clear


In [16]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,629
ABMD,565
ABNB,370
ADBE,0
ADYEY,4647


### Fidelity

In [17]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Sep-09-2022.csv'

In [18]:
!ls $DL

Portfolio_Positions_Sep-09-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_09.xls


In [19]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-01-", "-02-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,143.14,$1.00,$0.00,$143.14,NaN,NaN,NaN,NaN,0.37%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,3.00,$157.37,+$2.91,$472.11,+$8.73,+1.88%,-$40.62,-7.93%,1.22%,$512.73,$170.91,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$282.28,+$6.71,$1129.12,+$26.84,+2.43%,+$110.43,+10.84%,2.92%,$1018.69,$254.67,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$123.48,+$4.90,$246.96,+$9.80,+4.13%,-$38.54,-13.50%,0.64%,$285.50,$142.75,Cash
4,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$14.99,+$0.503,$194.87,+$6.53,+3.47%,-$19.82,-9.24%,0.50%,$214.69,$16.51,Cash


In [20]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,472
2,ROLLOVER IRA,ABMD,1129
3,ROLLOVER IRA,ABNB,247
4,ROLLOVER IRA,ADYEY,195
5,ROLLOVER IRA,AMD,1624


In [21]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,472
ABMD,1129
ABNB,247
ADYEY,195
AMD,1624


In [22]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [23]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,629,472,8026,0
ABMD,565,1129,2258,0
ABNB,370,247,494,0
ADBE,0,0,0,0
ADYEY,4647,195,2638,105


### TD Ameritrade

In [24]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [25]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [26]:
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [27]:
td = parse_file(path)
td.head()

,TD
AMEH,127.59
AMRK,377.65
APT,155.40
BYRN,70.00
EP,205.52


In [28]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,629,472,8026,0,0
ABMD,565,1129,2258,0,0
ABNB,370,247,494,0,0
ADBE,0,0,0,0,0
ADYEY,4647,195,2638,105,0


In [29]:
out.to_csv(f'{DL}/clean.csv')

In [30]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [15]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*